
## We’ll cover two key tools in your validator toolbox:

| Validator Type    | Scope        | Purpose                          |
| ----------------- | ------------ | -------------------------------- |
| `field_validator` | Single field | Validate or transform one field  |
| `model_validator` | Entire model | Validate or transform full model |

---

# 🧩 Part 1: `@field_validator`

## 📌 What is it?

`@field_validator` is a decorator in **Pydantic v2+** used to **add custom validation logic** for individual fields.

You use this when you want **fine control** over what values are allowed or how they are modified before becoming part of your model.

---

## 🧭 Why use it?

* Enforce rules not covered by built-in validators (like regex, gt/lt)
* Transform values (e.g., trim strings, convert formats)
* Validate complex formats (like license keys, custom IDs)

---

## 🎯 What can it do?

| Use Case              | Example                                    |
| --------------------- | ------------------------------------------ |
| Custom validation     | Enforce capital letters in name            |
| Format transformation | Strip spaces, normalize case               |
| Preprocess raw inputs | Parse a comma-separated string into a list |

---

## 🔥 Real-Life Scenarios

### ✅ 1. Clean up user input (remove leading/trailing spaces)

```python
from pydantic import BaseModel, field_validator

class User(BaseModel):
    username: str

    @field_validator("username")
    def trim_username(cls, v):
        return v.strip()
```

Now if the user sends `"   Mahbub   "`, it will become `"Mahbub"`.

---

### ✅ 2. Validate username format (must be alphanumeric)

```python
class User(BaseModel):
    username: str

    @field_validator("username")
    def alphanumeric_only(cls, v):
        if not v.isalnum():
            raise ValueError("Username must be alphanumeric")
        return v
```

---

### ✅ 3. Transform comma-separated string to list (e.g., skills)

```python
from typing import List

class Profile(BaseModel):
    skills: List[str]

    @field_validator("skills", mode="before")
    def parse_skills(cls, v):
        if isinstance(v, str):
            return [skill.strip() for skill in v.split(",")]
        return v
```

Now the user can send:

```json
{ "skills": "python, fastapi, sql" }
```

And the model will convert it to:

```python
["python", "fastapi", "sql"]
```

---

## ⚙️ Modes of `field_validator`

| Mode     | Description                            | Use Case                   |
| -------- | -------------------------------------- | -------------------------- |
| `before` | Runs **before** type coercion          | Cleaning raw user inputs   |
| `after`  | Runs **after** type coercion (default) | Validate final type, logic |

---

### 🔎 Example: Why mode matters

```python
from pydantic import BaseModel, field_validator

class Data(BaseModel):
    age: int

    @field_validator("age", mode="before")
    def reject_string_age(cls, v):
        if isinstance(v, str):
            raise ValueError("Age must not be a string!")
        return v
```

* If the mode is `"after"`, Pydantic will convert `"30"` (string) to `30` (int).
* With `"before"`, you **prevent** that conversion.

So `before` gives you **raw, untouched input** — useful when coercion is dangerous.

---

# 🧩 Part 2: `@model_validator`

## 📌 What is it?

`@model_validator` is for when you need to validate or manipulate **multiple fields at once** or **cross-field dependencies**.

Think of it like validating a whole form together instead of just one input box.

---

## 🧭 Why use it?

* To apply **business rules** across fields
* Validate when **two or more fields depend on each other**
* Dynamically compute or transform based on model context

---

## 🎯 What it can do?

| Use Case               | Example                                                  |
| ---------------------- | -------------------------------------------------------- |
| Ensure consistency     | Start date must be before end date                       |
| Derive field           | Generate full\_name from first + last name               |
| Validate conditionally | One field required **only if** another field has a value |

---

## 🔥 Real-Life Scenarios

### ✅ 1. Validate date range

```python
from datetime import date
from pydantic import BaseModel, model_validator

class Booking(BaseModel):
    start_date: date
    end_date: date

    @model_validator(mode='after')
    def check_date_range(self):
        if self.start_date >= self.end_date:
            raise ValueError("start_date must be before end_date")
        return self
```

---

### ✅ 2. Compute full name (data transformation)

```python
class User(BaseModel):
    first_name: str
    last_name: str
    full_name: str = ""

    @model_validator(mode="after")
    def build_full_name(self):
        self.full_name = f"{self.first_name} {self.last_name}"
        return self
```

Now `full_name` is automatically created.

---

### ✅ 3. Conditional requirement

```python
class ContactForm(BaseModel):
    email: str | None
    phone: str | None

    @model_validator(mode='after')
    def at_least_one_contact(self):
        if not (self.email or self.phone):
            raise ValueError("At least one contact method is required.")
        return self
```

This solves problems `field_validator` can’t—because it needs **both fields at once**.

---

## ⚙️ Modes in `model_validator`

| Mode     | Description                                              |
| -------- | -------------------------------------------------------- |
| `before` | Runs before type coercion                                |
| `after`  | Runs after all fields are parsed and validated (default) |

**Before** is rarely used, but useful when you receive weird JSON or deeply nested structures.

---

# 💡 Quick Recap: When to Use What?

| Task                                  | Use                                      |
| ------------------------------------- | ---------------------------------------- |
| Trim whitespace on strings            | `@field_validator` + `mode="before"`     |
| Enforce "start\_date < end\_date"     | `@model_validator`                       |
| Generate a `slug` from a `title`      | `@field_validator` or `@model_validator` |
| Validate `email or phone` must exist  | `@model_validator`                       |
| Reject `"30"` as valid for `age: int` | `@field_validator` + `mode="before"`     |

---
